# StepFunctionDelay

*  A small scale App to measure delay in Step Functions

In [5]:
import os
from io import BytesIO
import time
import zipfile
import numpy as np
import boto3
from tqdm import tqdm
from datetime import datetime, timezone

In [6]:
client = boto3.client('lambda')

In [7]:
function_prefix='StepFunctionDelay'

* All functions

In [4]:
functions=client.list_functions()
function_name=[item['FunctionName'] for item in functions['Functions']]
function_name

['serverlessrepo-DocumentGenerator-MyFunction-9MVH750993AD',
 'ServerlessAppPerfOpt_f5',
 'ServerlessAppPerfOpt_f4',
 'ServerlessAppPerfOpt_f6',
 'ServerlessAppPerfOpt_f2',
 'ServerlessAppPerfOpt_f3',
 'ServerlessAppPerfOpt_f1']

## Update all Functions in StepFunctionDelay

### Update Function Code

In [5]:
functions=[]
for file in os.listdir('functions'):
    path=os.path.abspath(os.path.join(os.path.dirname('__file__'), 'functions/'+file))
    if not file.startswith('.') and os.path.isdir(path):
        functions.append(file)
for function_folder in functions:
    buf = BytesIO()
    with zipfile.ZipFile(buf, 'w') as z:
        for file in os.listdir('functions/'+function_folder):
            z.write(os.path.abspath(os.path.join(os.path.dirname('__file__'), 'functions/{}/{}'.format(function_folder,file))), os.path.basename(os.path.join(os.path.dirname('__file__'), 'functions/{}/{}'.format(function_folder,file))))
    buf.seek(0)
    pkg = buf.read()
    client.update_function_code(FunctionName='{}_{}'.format(function_prefix, function_folder),ZipFile=pkg)

### Update Function Memory Configuration
* Available Memory Configurations: 128, 192, 256, 320, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024, 1088, 1152, 1216, 1280, 1344, 1408, 1472, 1536, 1600, 1664, 1728, 1792, 1856, 1920, 1984, 2048, 2112, 2176, 2240, 2304, 2368, 2432, 2496, 2560, 2624, 2688, 2752, 2816, 2880, 2944, 3008


In [6]:
mem_config_list={
    'f1':128,
    'f2':128,
}

In [7]:
for function in mem_config_list.keys():
    client.update_function_configuration(FunctionName='{}_{}'.format(function_prefix, function), MemorySize=mem_config_list[function])

## Execution

In [8]:
sfn_client = boto3.client('stepfunctions')

In [10]:
state_machines=sfn_client.list_state_machines()

In [12]:
state_machines['stateMachines']

[{'creationDate': datetime.datetime(2019, 12, 29, 22, 21, 52, 242000, tzinfo=tzlocal()),
  'name': 'HelloWorld',
  'stateMachineArn': 'arn:aws:states:us-east-2:499537426559:stateMachine:HelloWorld',
  'type': 'STANDARD'},
 {'creationDate': datetime.datetime(2019, 12, 29, 22, 32, 7, 221000, tzinfo=tzlocal()),
  'name': 'ServerlessAppPerfOpt_AppStepFunction',
  'stateMachineArn': 'arn:aws:states:us-east-2:499537426559:stateMachine:ServerlessAppPerfOpt_AppStepFunction',
  'type': 'EXPRESS'},
 {'creationDate': datetime.datetime(2019, 12, 30, 21, 56, 22, 479000, tzinfo=tzlocal()),
  'name': 'StepFunctionDelay',
  'stateMachineArn': 'arn:aws:states:us-east-2:499537426559:stateMachine:StepFunctionDelay',
  'type': 'EXPRESS'}]

In [15]:
stateMachineArn='arn:aws:states:us-east-2:499537426559:stateMachine:StepFunctionDelay'

### Execute StepFunctionDelay 100 times with 10-second interval

####  Execute StepFunctionDelay

In [18]:
for i in tqdm(range(100)):
    time.sleep(10)
    sfn_client.start_execution(stateMachineArn=stateMachineArn)

100%|██████████| 100/100 [16:54<00:00, 10.15s/it]


####  Execute App1

In [19]:
for i in tqdm(range(100)):
    time.sleep(10)
    sfn_client.start_execution(stateMachineArn='arn:aws:states:us-east-2:499537426559:stateMachine:ServerlessAppPerfOpt_AppStepFunction')

100%|██████████| 100/100 [16:53<00:00, 10.16s/it]


## CloudWatch Logs

In [8]:
logclient = boto3.client('logs')

In [9]:
def lambda_report_log_to_dict(log):
    res={}
    lis=[item.split(': ') for item in log.split('\t')]
    res['RequestId']=lis[0][1]
    res['Duration']=float(lis[1][1].split(' ')[0])
    res['Billed Duration']=int(lis[2][1].split(' ')[0])
    res['Memory Size']=int(lis[3][1].split(' ')[0])
    res['Max Memory Used']=int(lis[4][1].split(' ')[0])
    return res

### Query Step Function Logs

In [10]:
startTime_sfndelay=int(datetime.timestamp(datetime(year=2019,month=12,day=31,hour=5,minute=8,second=0,tzinfo=timezone.utc)))
endTime_sfndelay=int(datetime.timestamp(datetime(year=2019,month=12,day=31,hour=5,minute=26,second=0,tzinfo=timezone.utc)))

In [11]:
query_sfndelay = logclient.start_query(
    logGroupName='/aws/states/{}-Logs'.format(function_prefix),
    queryString="fields id, @timestamp| filter id = 4 or id = 5 or id = 9 or id = 10| sort id desc",
    startTime=startTime_sfndelay,
    endTime=endTime_sfndelay
)

In [12]:
query_results_sfndelay = logclient.get_query_results(
    queryId=query_sfndelay['queryId']
)

In [14]:
f1_starttimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfndelay['results'] if item[0]['value']=='4'])
f1_endtimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfndelay['results'] if item[0]['value']=='5'])
f2_starttimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfndelay['results'] if item[0]['value']=='9'])
f2_endtimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfndelay['results'] if item[0]['value']=='10'])

In [15]:
sfn_function_duration_f1=(f1_endtimestamp-f1_starttimestamp)*1000
sfn_function_duration_f2=(f2_endtimestamp-f2_starttimestamp)*1000

### Query Lambda Logs

In [16]:
query_lambdaduration_f1 = logclient.start_query(
    logGroupName='/aws/lambda/{}_{}'.format(function_prefix,'f1'),
    queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
    startTime=startTime_sfndelay,
    endTime=endTime_sfndelay
)
query_lambdaduration_f2 = logclient.start_query(
    logGroupName='/aws/lambda/{}_{}'.format(function_prefix,'f2'),
    queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
    startTime=startTime_sfndelay,
    endTime=endTime_sfndelay
)

In [17]:
query_results_lambdaduration_f1 = logclient.get_query_results(
    queryId=query_lambdaduration_f1['queryId']
)
query_results_lambdaduration_f2 = logclient.get_query_results(
    queryId=query_lambdaduration_f2['queryId']
)

In [18]:
f1_lamdaduration_list=[item['Duration'] for item in [lambda_report_log_to_dict(item [1]['value']) for item in query_results_lambdaduration_f1['results']]]
f2_lamdaduration_list=[item['Duration'] for item in [lambda_report_log_to_dict(item [1]['value']) for item in query_results_lambdaduration_f2['results']]]

### Calculate Delay Incurred by Step Function

#### Function 1

In [19]:
print('Number of Executions: ', len(f1_lamdaduration_list))
print('Average Duration Reported by Lambda: ', np.mean(f1_lamdaduration_list), 'ms')
print('Standard Deviation of Duration Reported by Lambda: ', np.std(f1_lamdaduration_list), 'ms')
print('Average Duration Reported by Step Function: ', np.mean(sfn_function_duration_f1), 'ms')
print('Standard Deviation of Duration Reported by Step Function: ', np.std(sfn_function_duration_f1), 'ms')
print('Average Discrepancy between Durations reported by Step Function and Lambda: ', np.mean(sfn_function_duration_f1-np.array(f1_lamdaduration_list)), 'ms')
print('Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda: ', np.std(sfn_function_duration_f1-np.array(f1_lamdaduration_list)), 'ms')

Number of Executions:  100
Average Duration Reported by Lambda:  404.033 ms
Standard Deviation of Duration Reported by Lambda:  4.402512805205684 ms
Average Duration Reported by Step Function:  428.12999963760376 ms
Standard Deviation of Duration Reported by Step Function:  11.390062562424406 ms
Average Discrepancy between Durations reported by Step Function and Lambda:  24.09699963760376 ms
Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda:  10.725752062646826 ms


#### Function 2

In [20]:
print('Number of Executions: ', len(f2_lamdaduration_list))
print('Average Duration Reported by Lambda: ', np.mean(f2_lamdaduration_list), 'ms')
print('Standard Deviation of Duration Reported by Lambda: ', np.std(f2_lamdaduration_list), 'ms')
print('Average Duration Reported by Step Function: ', np.mean(sfn_function_duration_f2), 'ms')
print('Standard Deviation of Duration Reported by Step Function: ', np.std(sfn_function_duration_f2), 'ms')
print('Average Discrepancy between Durations reported by Step Function and Lambda: ', np.mean(sfn_function_duration_f2-np.array(f2_lamdaduration_list)), 'ms')
print('Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda: ', np.std(sfn_function_duration_f2-np.array(f2_lamdaduration_list)), 'ms')

Number of Executions:  100
Average Duration Reported by Lambda:  203.3043 ms
Standard Deviation of Duration Reported by Lambda:  3.640076992317607 ms
Average Duration Reported by Step Function:  220.92000722885132 ms
Standard Deviation of Duration Reported by Step Function:  6.3713097529368286 ms
Average Discrepancy between Durations reported by Step Function and Lambda:  17.61570722885132 ms
Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda:  5.367679049879902 ms


### Calculate Delay Incurred by Step Function - App1

#### Query Step Function Logs

In [21]:
startTime_App1=int(datetime.timestamp(datetime(year=2019,month=12,day=31,hour=5,minute=32,second=0,tzinfo=timezone.utc)))
endTime_App1=int(datetime.timestamp(datetime(year=2019,month=12,day=31,hour=5,minute=49,second=0,tzinfo=timezone.utc)))
query_sfnApp1delay = logclient.start_query(
    logGroupName='/aws/states/{}-Logs'.format('ServerlessAppPerfOpt_AppStepFunction'),
    queryString="fields id, @timestamp| filter id = 4 or id = 5 or id = 9 or id = 10 or id = 16 or id = 19 or id = 20 or id = 22 or id = 28 or id = 29 or id = 33 or id = 34| sort id desc",
    startTime=startTime_App1,
    endTime=endTime_App1,
    limit=10000)

In [22]:
query_results_sfnApp1delay = logclient.get_query_results(
    queryId=query_sfnApp1delay['queryId']
)

In [23]:
App1_f1_starttimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='4'])
App1_f1_endtimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='5'])
App1_f2_starttimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='9'])
App1_f2_endtimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='10'])
App1_f3_starttimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='16'])
App1_f3_endtimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='20'])
App1_f4_starttimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='19'])
App1_f4_endtimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='22'])
App1_f5_starttimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='28'])
App1_f5_endtimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='29'])
App1_f6_starttimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='33'])
App1_f6_endtimestamp=np.sort([datetime.timestamp(datetime.strptime(item[1]['value'], '%Y-%m-%d %H:%M:%S.%f')) for item in query_results_sfnApp1delay['results'] if item[0]['value']=='34'])
sfn_function_duration_App1_f1=(App1_f1_endtimestamp-App1_f1_starttimestamp)*1000
sfn_function_duration_App1_f2=(App1_f2_endtimestamp-App1_f2_starttimestamp)*1000
sfn_function_duration_App1_f3=(App1_f3_endtimestamp-App1_f3_starttimestamp)*1000
sfn_function_duration_App1_f4=(App1_f4_endtimestamp-App1_f4_starttimestamp)*1000
sfn_function_duration_App1_f5=(App1_f5_endtimestamp-App1_f5_starttimestamp)*1000
sfn_function_duration_App1_f6=(App1_f6_endtimestamp-App1_f6_starttimestamp)*1000

#### Query Lambda Logs

In [24]:
query_lambdaduration_App1_f1 = logclient.start_query(
    logGroupName='/aws/lambda/{}_{}'.format('ServerlessAppPerfOpt','f1'),
    queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
    startTime=startTime_App1,
    endTime=endTime_App1
)


In [25]:
query_lambdaduration_App1_f2 = logclient.start_query(
    logGroupName='/aws/lambda/{}_{}'.format('ServerlessAppPerfOpt','f2'),
    queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
    startTime=startTime_App1,
    endTime=endTime_App1
)


In [26]:
query_lambdaduration_App1_f3 = logclient.start_query(
    logGroupName='/aws/lambda/{}_{}'.format('ServerlessAppPerfOpt','f3'),
    queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
    startTime=startTime_App1,
    endTime=endTime_App1
)


In [27]:
query_lambdaduration_App1_f4 = logclient.start_query(
    logGroupName='/aws/lambda/{}_{}'.format('ServerlessAppPerfOpt','f4'),
    queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
    startTime=startTime_App1,
    endTime=endTime_App1
)


In [28]:
query_lambdaduration_App1_f5 = logclient.start_query(
    logGroupName='/aws/lambda/{}_{}'.format('ServerlessAppPerfOpt','f5'),
    queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
    startTime=startTime_App1,
    endTime=endTime_App1
)


In [29]:
query_lambdaduration_App1_f6 = logclient.start_query(
    logGroupName='/aws/lambda/{}_{}'.format('ServerlessAppPerfOpt','f6'),
    queryString="fields @timestamp, @message| filter @message like 'REPORT'| sort @timestamp asc",
    startTime=startTime_App1,
    endTime=endTime_App1
)


In [30]:
query_results_lambdaduration_App1_f1 = logclient.get_query_results(
    queryId=query_lambdaduration_App1_f1['queryId']
)
App1_f1_lamdaduration_list=[item['Duration'] for item in [lambda_report_log_to_dict(item [1]['value']) for item in query_results_lambdaduration_App1_f1['results']]]
query_results_lambdaduration_App1_f2 = logclient.get_query_results(
    queryId=query_lambdaduration_App1_f2['queryId']
)
App1_f2_lamdaduration_list=[item['Duration'] for item in [lambda_report_log_to_dict(item [1]['value']) for item in query_results_lambdaduration_App1_f2['results']]]
query_results_lambdaduration_App1_f3 = logclient.get_query_results(
    queryId=query_lambdaduration_App1_f3['queryId']
)
App1_f3_lamdaduration_list=[item['Duration'] for item in [lambda_report_log_to_dict(item [1]['value']) for item in query_results_lambdaduration_App1_f3['results']]]
query_results_lambdaduration_App1_f4 = logclient.get_query_results(
    queryId=query_lambdaduration_App1_f4['queryId']
)
App1_f4_lamdaduration_list=[item['Duration'] for item in [lambda_report_log_to_dict(item [1]['value']) for item in query_results_lambdaduration_App1_f4['results']]]
query_results_lambdaduration_App1_f5 = logclient.get_query_results(
    queryId=query_lambdaduration_App1_f5['queryId']
)
App1_f5_lamdaduration_list=[item['Duration'] for item in [lambda_report_log_to_dict(item [1]['value']) for item in query_results_lambdaduration_App1_f5['results']]]
query_results_lambdaduration_App1_f6 = logclient.get_query_results(
    queryId=query_lambdaduration_App1_f6['queryId']
)
App1_f6_lamdaduration_list=[item['Duration'] for item in [lambda_report_log_to_dict(item [1]['value']) for item in query_results_lambdaduration_App1_f6['results']]]

#### Function 1

In [31]:
print('Number of Executions: ', len(App1_f1_lamdaduration_list))
print('Average Duration Reported by Lambda: ', np.mean(App1_f1_lamdaduration_list), 'ms')
print('Standard Deviation of Duration Reported by Lambda: ', np.std(App1_f1_lamdaduration_list), 'ms')
print('Average Duration Reported by Step Function: ', np.mean(sfn_function_duration_App1_f1), 'ms')
print('Standard Deviation of Duration Reported by Step Function: ', np.std(sfn_function_duration_App1_f1), 'ms')
print('Average Discrepancy between Durations reported by Step Function and Lambda: ', np.mean(sfn_function_duration_App1_f1-np.array(App1_f1_lamdaduration_list)), 'ms')
print('Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda: ', np.std(sfn_function_duration_App1_f1-np.array(App1_f1_lamdaduration_list)), 'ms')

Number of Executions:  100
Average Duration Reported by Lambda:  403.7752 ms
Standard Deviation of Duration Reported by Lambda:  4.185725380385101 ms
Average Duration Reported by Step Function:  430.18998622894287 ms
Standard Deviation of Duration Reported by Step Function:  35.98907378409499 ms
Average Discrepancy between Durations reported by Step Function and Lambda:  26.41478622894288 ms
Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda:  35.996724710417524 ms


#### Function 2

In [32]:
print('Number of Executions: ', len(App1_f2_lamdaduration_list))
print('Average Duration Reported by Lambda: ', np.mean(App1_f2_lamdaduration_list), 'ms')
print('Standard Deviation of Duration Reported by Lambda: ', np.std(App1_f2_lamdaduration_list), 'ms')
print('Average Duration Reported by Step Function: ', np.mean(sfn_function_duration_App1_f2), 'ms')
print('Standard Deviation of Duration Reported by Step Function: ', np.std(sfn_function_duration_App1_f2), 'ms')
print('Average Discrepancy between Durations reported by Step Function and Lambda: ', np.mean(sfn_function_duration_App1_f2-np.array(App1_f2_lamdaduration_list)), 'ms')
print('Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda: ', np.std(sfn_function_duration_App1_f2-np.array(App1_f2_lamdaduration_list)), 'ms')

Number of Executions:  100
Average Duration Reported by Lambda:  203.03099999999998 ms
Standard Deviation of Duration Reported by Lambda:  3.0641339722668786 ms
Average Duration Reported by Step Function:  220.72999715805054 ms
Standard Deviation of Duration Reported by Step Function:  20.476272049985642 ms
Average Discrepancy between Durations reported by Step Function and Lambda:  17.69899715805054 ms
Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda:  20.330270199515617 ms


#### Function 3

In [33]:
print('Number of Executions: ', len(App1_f3_lamdaduration_list))
print('Average Duration Reported by Lambda: ', np.mean(App1_f3_lamdaduration_list), 'ms')
print('Standard Deviation of Duration Reported by Lambda: ', np.std(App1_f3_lamdaduration_list), 'ms')
print('Average Duration Reported by Step Function: ', np.mean(sfn_function_duration_App1_f3), 'ms')
print('Standard Deviation of Duration Reported by Step Function: ', np.std(sfn_function_duration_App1_f3), 'ms')
print('Average Discrepancy between Durations reported by Step Function and Lambda: ', np.mean(sfn_function_duration_App1_f3-np.array(App1_f3_lamdaduration_list)), 'ms')
print('Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda: ', np.std(sfn_function_duration_App1_f3-np.array(App1_f3_lamdaduration_list)), 'ms')

Number of Executions:  100
Average Duration Reported by Lambda:  804.2271000000001 ms
Standard Deviation of Duration Reported by Lambda:  4.207432541348701 ms
Average Duration Reported by Step Function:  820.3500175476074 ms
Standard Deviation of Duration Reported by Step Function:  19.749113067043325 ms
Average Discrepancy between Durations reported by Step Function and Lambda:  16.122917547607422 ms
Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda:  19.316455176125064 ms


#### Function 4

In [34]:
print('Number of Executions: ', len(App1_f4_lamdaduration_list))
print('Average Duration Reported by Lambda: ', np.mean(App1_f4_lamdaduration_list), 'ms')
print('Standard Deviation of Duration Reported by Lambda: ', np.std(App1_f4_lamdaduration_list), 'ms')
print('Average Duration Reported by Step Function: ', np.mean(sfn_function_duration_App1_f4), 'ms')
print('Standard Deviation of Duration Reported by Step Function: ', np.std(sfn_function_duration_App1_f4), 'ms')
print('Average Discrepancy between Durations reported by Step Function and Lambda: ', np.mean(sfn_function_duration_App1_f4-np.array(App1_f4_lamdaduration_list)), 'ms')
print('Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda: ', np.std(sfn_function_duration_App1_f4-np.array(App1_f4_lamdaduration_list)), 'ms')

Number of Executions:  100
Average Duration Reported by Lambda:  1003.6695000000001 ms
Standard Deviation of Duration Reported by Lambda:  3.2994482493289725 ms
Average Duration Reported by Step Function:  1022.1100091934204 ms
Standard Deviation of Duration Reported by Step Function:  21.139969033778595 ms
Average Discrepancy between Durations reported by Step Function and Lambda:  18.440509193420407 ms
Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda:  20.890728082924664 ms


#### Function 5

In [35]:
print('Number of Executions: ', len(App1_f5_lamdaduration_list))
print('Average Duration Reported by Lambda: ', np.mean(App1_f5_lamdaduration_list), 'ms')
print('Standard Deviation of Duration Reported by Lambda: ', np.std(App1_f5_lamdaduration_list), 'ms')
print('Average Duration Reported by Step Function: ', np.mean(sfn_function_duration_App1_f5), 'ms')
print('Standard Deviation of Duration Reported by Step Function: ', np.std(sfn_function_duration_App1_f5), 'ms')
print('Average Discrepancy between Durations reported by Step Function and Lambda: ', np.mean(sfn_function_duration_App1_f5-np.array(App1_f5_lamdaduration_list)), 'ms')
print('Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda: ', np.std(sfn_function_duration_App1_f5-np.array(App1_f5_lamdaduration_list)), 'ms')

Number of Executions:  100
Average Duration Reported by Lambda:  503.8387 ms
Standard Deviation of Duration Reported by Lambda:  3.996418810635337 ms
Average Duration Reported by Step Function:  520.8300065994263 ms
Standard Deviation of Duration Reported by Step Function:  19.920870145359665 ms
Average Discrepancy between Durations reported by Step Function and Lambda:  16.991306599426267 ms
Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda:  19.67841544632416 ms


#### Function 6

In [36]:
print('Number of Executions: ', len(App1_f6_lamdaduration_list))
print('Average Duration Reported by Lambda: ', np.mean(App1_f6_lamdaduration_list), 'ms')
print('Standard Deviation of Duration Reported by Lambda: ', np.std(App1_f6_lamdaduration_list), 'ms')
print('Average Duration Reported by Step Function: ', np.mean(sfn_function_duration_App1_f6), 'ms')
print('Standard Deviation of Duration Reported by Step Function: ', np.std(sfn_function_duration_App1_f6), 'ms')
print('Average Discrepancy between Durations reported by Step Function and Lambda: ', np.mean(sfn_function_duration_App1_f6-np.array(App1_f6_lamdaduration_list)), 'ms')
print('Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda: ', np.std(sfn_function_duration_App1_f6-np.array(App1_f6_lamdaduration_list)), 'ms')

Number of Executions:  100
Average Duration Reported by Lambda:  603.2264 ms
Standard Deviation of Duration Reported by Lambda:  2.6983763710794637 ms
Average Duration Reported by Step Function:  620.4499983787537 ms
Standard Deviation of Duration Reported by Step Function:  24.042197570967733 ms
Average Discrepancy between Durations reported by Step Function and Lambda:  17.22359837875366 ms
Standard Deviation of Discrepancy between Durations reported by Step Function and Lambda:  23.9623726378212 ms


In [40]:
discrepancy_App1_f1 = sfn_function_duration_App1_f1-np.array(App1_f1_lamdaduration_list)
discrepancy_App1_f2 = sfn_function_duration_App1_f2-np.array(App1_f2_lamdaduration_list)
discrepancy_App1_f3 = sfn_function_duration_App1_f3-np.array(App1_f3_lamdaduration_list)
discrepancy_App1_f4 = sfn_function_duration_App1_f4-np.array(App1_f4_lamdaduration_list)
discrepancy_App1_f5 = sfn_function_duration_App1_f5-np.array(App1_f5_lamdaduration_list)
discrepancy_App1_f6 = sfn_function_duration_App1_f6-np.array(App1_f6_lamdaduration_list)

In [44]:
discrepancy_App1 = np.concatenate((discrepancy_App1_f1, discrepancy_App1_f2, discrepancy_App1_f3, discrepancy_App1_f4, discrepancy_App1_f5, discrepancy_App1_f6))

In [46]:
np.mean(discrepancy_App1) # AWS Step Functions Interaction Delay

18.815352517700198